# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.environment import Environment

In [2]:
from azureml.core import Workspace, Experiment

In [3]:
Environment(name="myenv")

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210714.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
        

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

In [8]:
env = Environment.get(workspace = ws, name = "AzureML-Tutorial")

In [9]:

# env = Environment.get(workspace = ws, name = "AzureML-SKLearn-0.24-ubuntu18.04-py37-cpu")

In [10]:
experiment_name = 'CapstoneProject_HyperDrive'

experiment=Experiment(ws, experiment_name)

In [11]:
print(ws.name)

quick-starts-ws-155734


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "UdacityProject"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        'C': uniform(0.001, 1.0), # Regularization
        'max_iter': choice(5, 20, 100, 200) # Max number of iterations (aka epochs)
    }
)

# if "training" not in os.listdir():
#     os.mkdir("./training")
    
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', 
   compute_target=cpu_cluster,
   entry_script='train.py',
   conda_packages=['scikit-learn==0.22.2.post1'])

hyperdrive_run_config = HyperDriveConfig(run_config=src,
#     estimator = estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4,
#                                         environment = env)

SyntaxError: unexpected EOF while parsing (<ipython-input-28-a8050883c423>, line 37)

In [23]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [37]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')


from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
#                       arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

In [38]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        'C': uniform(0.001, 1.0), # Regularization
        'max_iter': choice(5, 20, 100, 200) # Max number of iterations (aka epochs)
    }
)

# if "training" not in os.listdir():
#     os.mkdir("./training")
    
# #TODO: Create your estimator and hyperdrive config
# estimator = SKLearn(source_directory='.', 
#    compute_target=cpu_cluster,
#    entry_script='train.py',
#    conda_packages=['scikit-learn==0.22.2.post1'])

hyperdrive_run_config = HyperDriveConfig(run_config=src,
#     estimator = estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)
#                                         

In [39]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [40]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
# hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [45]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n learning rate:',parameter_values[3])



Best Run Id:  HD_4349bd79-2b5f-4768-a0e6-979486d67d6c_0

 learning rate: 100


In [49]:
parameter_values

['--C', '0.2937809463771922', '--max_iter', '100']

In [50]:
best_run_metrics

{'Regularization Strength:': 0.2937809463771922,
 'Max iterations:': 100,
 'Accuracy': 0.8833333333333333}

In [46]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject_HyperDrive,HD_4349bd79-2b5f-4768-a0e6-979486d67d6c_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [57]:
best_run.download_file('outputs', 'score.py')

UserErrorException: UserErrorException:
	Message: File with path outputs was not found,
available files include: azureml-logs/55_azureml-execution-tvmps_62f57f4e4ccd13d4b34040ea5f5fb97d2009a09f0e9dbcdbb0a54208b33c7aec_d.txt,azureml-logs/65_job_prep-tvmps_62f57f4e4ccd13d4b34040ea5f5fb97d2009a09f0e9dbcdbb0a54208b33c7aec_d.txt,azureml-logs/70_driver_log.txt,azureml-logs/75_job_post-tvmps_62f57f4e4ccd13d4b34040ea5f5fb97d2009a09f0e9dbcdbb0a54208b33c7aec_d.txt,azureml-logs/process_info.json,azureml-logs/process_status.json,logs/azureml/93_azureml.log,logs/azureml/dataprep/backgroundProcess.log,logs/azureml/dataprep/backgroundProcess_Telemetry.log,logs/azureml/job_prep_azureml.log,logs/azureml/job_release_azureml.log,outputs/model.joblib.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path outputs was not found,\navailable files include: azureml-logs/55_azureml-execution-tvmps_62f57f4e4ccd13d4b34040ea5f5fb97d2009a09f0e9dbcdbb0a

In [55]:
best_run.download_file('outputs/model.joblib', 'envFile.yml')

In [56]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
# model = run.register_model(model_name = "heartfailuremodel", model_path = "heart_failure_model.pkl")

# Combine scoring script & environment in Inference configuration

inference_config = InferenceConfig(entry_script='score.py', environment=sklearn_env)
# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                      auth_enabled=True)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "my_web_service",
                       models = [best_model],
                       inference_config = inference_config,
                       deployment_config = deployment_config,
                      overwrite=True)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service